In [67]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import *
import numpy as np
from sklearn.decomposition import *
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier

In [68]:
pd.options.display.max_columns = 999
pd.options.display.max_rows = 60

In [69]:
##Define the object for the season team
class MyDf:
    def __init__(self, mydf):
        self.df = mydf 

In [70]:
def select_season (df, season):
    return df[df['season']==season]

In [71]:
def cut_out_n_m_matches(df,n,m):
    a=df['H_ROUND'].unique()[n-1]
    b=df['H_ROUND'].unique()[-m]
    
    return df[(df['H_ROUND']>a) & (df['H_ROUND']<b)]
    

In [72]:
def train_test_index(df,predicted_season):
    train_index=df[df.season!=predicted_season].index
    test_index=df[df.season==predicted_season].index
    return train_index, test_index

In [73]:
def train_test_index_only_round(df,predicted_round):
    train_index=df[df['H_ROUND']<predicted_round].index
    test_index=df[df['H_ROUND']==predicted_round].index
    return train_index, test_index

In [74]:
def split_train_test_data(df,train_index,test_index,col_train,col_pred,i):
        
    X_train=df.loc[train_index].iloc[:,col_train]
    Y_train=df.loc[train_index][col_pred]

   
    
    X_test=df.loc[test_index]
    Y_test=df.loc[test_index]

    X_test=X_test[X_test.H_ROUND==i].iloc[:,col_train]
    Y_test=Y_test[Y_test.H_ROUND==i][col_pred]

    return X_train, Y_train, X_test, Y_test

In [75]:
def Predict_whole_season_by_one (BIG, predseason, col_train, col_pred, method, method_name):
        
    tmp=pd.DataFrame()
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]
    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        #print(method.score(X_test, Y_test))
        #print('hi')
        lis.append(float(method.score(X_test, Y_test)))

    tmp[method_name]=lis 
    
    return tmp

In [76]:
def Predict_whole_season_with_one (BIG, predseason, col_train, col_pred,method, method_name):
        
    tmp=pd.DataFrame()    
    
    train_index, test_index = train_test_index(BIG,predseason)
    lis=[]

    
    for i in BIG[BIG.season==predseason].H_ROUND.unique():
       
        X_train,Y_train,X_test,Y_test = split_train_test_data(BIG,train_index,test_index,col_train,col_pred,i) 
        method.fit(X_train,Y_train)
        lis.append(float(method.score(X_test, Y_test)))
        
        train_index=list(train_index)+list(X_test.index)
       
    tmp[method_name]=lis     
    return tmp
    

In [77]:
def Predict_season_with_current_season (df, predseason, col_train, col_pred,method, method_name):
    
    tmp=pd.DataFrame()
    tmp_2=df[df['season']==predseason]
    lis=[]
    
    second_round=tmp_2['H_ROUND'].unique()[1]
    
        
    for i in range(int(df.H_ROUND.unique()[1]), int(df.H_ROUND.unique()[-1])):
        
        train_index, test_index = train_test_index_only_round(df,i)
        
        
        
        X_train=df.loc[train_index].iloc[:,col_train]
        Y_train=df.loc[train_index][col_pred]

        
        X_test=df.loc[test_index].iloc[:,col_train]
        Y_test=df.loc[test_index][col_pred]
        
       
        method.fit(X_train,Y_train)
        
        lis.append(float(method.score(X_test, Y_test)))
        
       
    tmp[method_name]=lis     
    
    return tmp


In [78]:
def find_best_model (X,col_pred,col_train,method):
    n=100
    from sklearn.cross_validation import train_test_split
    tmp=0
    for i in range(n):
        X_train, X_test, y_train, y_test = train_test_split(X.iloc[:,column_to_use_as_prediction], X[col_pred], test_size=0.20)
        method.fit(X_train,y_train)
        if method.score(X_test,y_test)>tmp:
            Best=method.fit(X_train,y_train)
    return Best

# Loading data

In [79]:
final=pd.read_csv('../Data/from_sqlite_Data/realfinal.csv')
final.drop('Unnamed: 0',axis=1,inplace=True)
allchampionships=[]

In [80]:
for i in range(len(final.country_id.unique())):
    allchampionships.append(MyDf(final[final['country_id']==final.country_id.unique()[i]].sort(columns='Date')))
    allchampionships[i].df.reset_index(inplace=True)
    allchampionships[i].df.drop(['index'],axis=1,inplace=True)

/Users/nicolamacchitella/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


# Cut out first and last n,m matches

In [81]:
n=4
m=4

In [82]:
for i in range(len(final.country_id.unique())):
    allchampionships[i].df=cut_out_n_m_matches(allchampionships[i].df,n,m)
    

In [83]:
pd.DataFrame(allchampionships[0].df.columns)

,0
0,country_id
1,season
2,Date
3,HomeTeam
4,AwayTeam
5,FTHG
6,FTAG
7,FTR
8,H_ROUND
9,H_TP3


## SVM_LINEAR

In [98]:
season_to_be_predicted='2015/2016'
column_to_use_as_prediction=[54,55]
column_to_predict='Goal/No_Goal'

In [108]:
mix=[allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict]

In [116]:
svm_linear = svm.SVC(kernel='linear')
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [117]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_linear,'svm_linear')

In [118]:
#c=Predict_season_with_current_season(allchampionships[0].df,season_to_be_predicted, column_to_use_as_prediction, column_to_predict,svm_linear,'svm_linear')

In [119]:
a['svm_linear'].mean()

0.601955782312925

In [121]:
b['svm_linear'].mean()

0.601955782312925

## SVM_RBF


In [110]:
svm_rbf = svm.SVC(kernel='rbf', gamma = 100)
a=Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict,svm_rbf,'svm_rbf')

In [111]:
b=Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, svm_rbf,'svm_rbf')

In [113]:
a['svm_rbf'].mean()

0.5593537414965987

In [114]:
b['svm_rbf'].mean()

0.5599489795918368

## CART

In [ ]:
dtc = DecisionTreeClassifier()
Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

In [ ]:
Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, dtc,'dtc')

# Adaboost Classifier 

In [ ]:
bagb = AdaBoostClassifier(DecisionTreeClassifier(max_depth=10), n_estimators=20)
Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')

In [ ]:
Predict_whole_season_with_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, bagb,'bagb')

# Gradient Boosting Classifier


In [ ]:
gradb = GradientBoostingClassifier(max_depth=10, n_estimators=100)
Predict_whole_season_by_one(allchampionships[0].df,season_to_be_predicted,column_to_use_as_prediction,column_to_predict, gradb,'gradb')

In [ ]:
Predict_whole_season_with_one(Italy,'2015/2016',[46,47], 'FTR', gradb,'gradb')

# Random Forest

In [ ]:
rdf = RandomForestClassifier(max_depth=10, n_estimators=50)
Predict_whole_season_by_one(Italy,'2015/2016',[46,47], 'FTR', rdf,'rdf')

In [ ]:
Predict_whole_season_with_one(Italy,'2015/2016',[46,47], 'FTR', rdf,'rdf')

In [294]:
a=select_season(allchampionships[0].df, '2015/2016')

In [298]:
a.H_ROUND.unique()

array([  5.,   6.,   7.,   8.,   9.,  10.,  11.,  12.,  13.,  14.,  15.,
        16.,  17.,  18.,  19.,  20.,  21.,  22.,  23.,  24.,  25.,  26.,
        27.,  28.,  29.,  30.,  31.,  32.])

In [356]:
g=30

In [357]:
Y_test=a[a.H_ROUND>g]['FTR']

In [358]:
X_test=a[a.H_ROUND>g].iloc[:,[50,51]]

In [359]:
Y_train=a[a.H_ROUND<=g]['FTR']
X_train=a[a.H_ROUND<=g].iloc[:,[50,51]]

In [360]:
svm_linear.fit(X_train,Y_train,)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [366]:
svm_linear.decision_function(X_test)

array([[ 0.69684217, -0.2480954 , -1.00002401],
       [ 0.15444954, -1.41276864, -0.99991077],
       [ 0.44449283, -0.33884951, -1.00003654],
       [ 0.21196258, -0.92246081, -0.9999758 ],
       [ 0.01261307, -0.94118426, -0.99999336],
       [ 0.24756597, -0.79192232, -0.99999106],
       [ 0.67845892,  0.4757038 , -1.00013051],
       [ 0.23072064, -0.48317437, -1.0000374 ],
       [ 0.78709045, -0.10184035, -1.00003598],
       [ 0.64821307, -0.13351353, -1.00004551]])